In [ ]:
fs = 400000


In [ ]:
import csv
import numpy as np
real_datas = []
with open('cyclo2_fresh.csv', newline='') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        real_datas.append( list( map(float,row) ) )
    csvfile.close()
real_datas = np.array(real_datas)

In [ ]:
# signal
import matplotlib.pyplot as plt
from scipy import stats
print( stats.describe(real_datas[3]) )
plt.figure(figsize=(20,10))
plt.plot(np.arange(0,len(real_datas[1])/fs,1/fs),real_datas[10])
plt.xlabel("Time")
plt.ylabel("Amp")

In [ ]:
# Zxx (頻率軸,時域軸)
# 頻率軸 index = 0 (頻率最高) -> index = last (頻率最低)
# 時域軸 index = 0 (時間起點) -> index = last (時間終點)

from scipy import signal
fs = 400000
freq, times, Zxx = signal.stft(real_datas[0], fs=fs, window='blackmanharris', nperseg=256, noverlap=128)
print(freq.shape)
print(times.shape)
print(Zxx.shape)

a11 = signal.istft(Zxx, fs=fs, window='blackmanharris', nperseg=256, noverlap=128)
print(type(a11[1]))
print("length={%d}"%a11[1].shape)

freq = freq[:-1]
times = times[:-1]
Zxx = Zxx[1:,:-1]
print(freq.shape)
print(times.shape)
print(Zxx.shape)

# print(freq/1000)
# print(times)


In [ ]:
print("raw:")
print(np.max(abs(Zxx)))
print(np.min(abs(Zxx)))
print(np.mean(abs(Zxx)))

k = np.log10(abs(Zxx))

print("log:")
print(np.max(k))
print(np.min(k))
print(np.mean(k))

print("nor:")
tune = 4.2

k_ = (k+tune)/tune
print(np.max(k_))
print(np.min(k_))
print(np.mean(k_))

In [ ]:
tune = 4.2
for i in range(1,15000,50):
    freq, times, Zxx = signal.stft(real_datas[i], fs=fs, window='blackmanharris', nperseg=256, noverlap=128)
    k = np.log10(abs(Zxx))
    k_ = (k+tune)/tune
    print(np.max(k_),np.min(k_))

In [ ]:
# draw stft
from scipy import signal

import matplotlib.pyplot as plt
cm=plt.cm.get_cmap('plasma')
plt.figure(figsize=(20,10))

plt.pcolormesh(times, freq/1000, 20*np.log10(abs(Zxx)), cmap=cm)
plt.colorbar()
plt.xlabel("Time(s)",fontsize=16)
plt.ylabel("Frequency(kHz)",fontsize=16)

In [ ]:
# training start
import Ipynb_importer
# %load train.py
import torch
from torch import autograd
from torch import optim
import json
#from utils import save_samples
import numpy as np
import pprint
import pickle
import datetime
#from dcgan import *
from utils import *
from logger import *
cuda = True if torch.cuda.is_available() else False

In [ ]:
# =============Logger===============
LOGGER = logging.getLogger('dcgan')
LOGGER.setLevel(logging.DEBUG)

LOGGER.info('Initialized logger.')
init_console_logger(LOGGER)

# =============Parameters===============
args = parse_arguments()
epochs = args['num_epochs']
batch_size = args['batch_size']
latent_dim = args['latent_dim']
ngpus = args['ngpus']
model_size = args['model_size']
model_dir = make_path(os.path.join(args['output_dir'],
                                   datetime.datetime.now().strftime("%Y%m%d%H%M%S")))
args['model_dir'] = model_dir

# save samples for every N epochs.
epochs_per_sample = args['epochs_per_sample']

# gradient penalty regularization factor.
lmbda = args['lmbda']

# Dir
audio_dir = args['audio_dir']
output_dir = args['output_dir']

In [ ]:
# # test generator
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.utils.data
# import numpy as np

# nc = 1
# nz = 100
# ngf = 64
# ndf = 64
# model_size = 4

# # 16 = (8-1)*2 - 2*0 + 1*(4-1) + 0 + 1
# conv = nn.ConvTranspose2d( model_size * 16, model_size * 8, 4, 2, 1, bias=False)
# conv1 = nn.ConvTranspose2d( model_size * 8, model_size * 4, 4, 2, 1, bias=False)
# conv2 = nn.ConvTranspose2d( model_size * 4, model_size * 2, 4, 2, 1, bias=False)
# conv3 = nn.ConvTranspose2d( model_size * 2, nc, 3, 1, 1, bias=False) # 

# a = torch.tensor(np.ones((1,8,8,64)),dtype=torch.float32)
# a = a.permute(0,3,1,2)
# print(a.shape)
# a = conv(a)
# print(a.shape)
# a = conv1(a)
# print(a.shape)
# a = conv2(a)
# print(a.shape)
# a = conv3(a)
# print(a.shape)

In [ ]:
# # # test discriminator
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.utils.data
# import numpy as np

# nc = 1
# nz = 100
# model_size = 4

# # 16 = (8-1)*2 - 2*0 + 1*(4-1) + 0 + 1
# conv = nn.Conv2d(nc, model_size*8, 4, 2, 1, bias=False)
# leaky = nn.LeakyReLU(0.2, inplace=True)
# drop = nn.Dropout(p=0.25)
# conv1 = nn.Conv2d(model_size*8, model_size*16, 4, 2, 1, bias=False)
# conv2 = nn.Conv2d(model_size*16, model_size*32, 4, 2, 1, bias=False)
# conv3 = nn.Conv2d(model_size*32, model_size*64, 4, 2, 1, bias=False)

# a = torch.tensor(np.ones((2,64,64,1)),dtype=torch.float32)
# a = a.permute(0,3,1,2)
# print(type(a))
# print(a.shape)
# a = drop(leaky(conv(a)))
# print(a.shape)
# a = conv1(a)
# print(a.shape)
# a = conv2(a)
# print(a.shape)
# a = conv3(a)
# print(a.shape)

In [ ]:
import Ipynb_importer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import numpy as np

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 128

# Size of feature maps in discriminator
ndf = 128

model_size = 64


class Generator(nn.Module):
    def __init__(self, ngpu, verbose = False):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.verbose = verbose
        self.fc1 = nn.Linear(nz, ndf*ndf) 
        self.dconv1 = nn.Sequential(
            nn.ReLU(True),
            nn.ConvTranspose2d( model_size * 16, model_size * 8, 4, 2, 1, bias=True),
            nn.BatchNorm2d(model_size * 8),
            nn.ReLU(True)
        )
        self.dconv2 = nn.Sequential(
            nn.ConvTranspose2d( model_size * 8, model_size * 4, 4, 2, 1, bias=True),
            nn.BatchNorm2d(model_size * 4),
            nn.ReLU(True),
        )
        self.dconv3 = nn.Sequential(
            nn.ConvTranspose2d(model_size * 4, model_size * 2, 4, 2, 1, bias=True),
            nn.BatchNorm2d(model_size * 2),
            nn.ReLU(True),
        )
        self.dconv4 = nn.Sequential(
            nn.ConvTranspose2d(model_size * 2 , model_size , 4, 2, 1, bias=True),
            nn.BatchNorm2d(model_size),
            nn.ReLU(True),
        )
        self.dconv5 = nn.Sequential(
            nn.ConvTranspose2d( model_size , nc, 4, 2, 1, bias=True),
            nn.Tanh()
        )

    def forward(self, x):
        if(self.verbose):
            print(x.shape)
            
        x = self.fc1(x)
        x =  x.view(-1,1024,4,4)
        if(self.verbose):
            print(x.shape)
            
        x = self.dconv1(x)
        if(self.verbose):
            print(x.shape)
            
        x = self.dconv2(x)
        if(self.verbose):
            print(x.shape)
            
        x = self.dconv3(x)
        if(self.verbose):
            print(x.shape)
            
        x = self.dconv4(x)
        if(self.verbose):
            print(x.shape)
            
        x = self.dconv5(x)
        if(self.verbose):
            print(x.shape)
            
        return x


class Discriminator(nn.Module):
    def __init__(self, ngpu, verbose=False):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.verbose = verbose
        self.conv1 = nn.Sequential(
            nn.Conv2d(nc, model_size, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.25)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(model_size, model_size*2, 4, 2, 1, bias=True),
            nn.BatchNorm2d(model_size*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.25)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(model_size*2, model_size*4, 4, 2, 1, bias=True),
            nn.BatchNorm2d(model_size*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.25)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(model_size*4, model_size*8, 4, 2, 1, bias=True),
            nn.BatchNorm2d(model_size*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.25)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(model_size*8, model_size*16, 4, 2, 1, bias=True),
            nn.BatchNorm2d(model_size*16),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.25)
        )
        
        self.fc2 = nn.Linear(16384, 1)
        self.sig = nn.Sigmoid()
        
    def forward(self, x):
        if (self.verbose):
            print(x.shape)
        x =  self.conv1(x)
        if (self.verbose):
            print(x.shape)
        x =  self.conv2(x)
        if (self.verbose):
            print(x.shape)
        x =  self.conv3(x)
        if (self.verbose):
            print(x.shape)
        x =  self.conv4(x)
        if (self.verbose):
            print(x.shape)
        x =  self.conv5(x)
        if (self.verbose):
            print(x.shape)
        x =  x.view(-1,16384)
        if (self.verbose):
            print(x.shape)
        x =  self.sig( self.fc2( x ) )
        if (self.verbose):
            print(x.shape)
        return  x

In [ ]:
# test netG
ngpu = 1
netG = Generator(ngpu,verbose=True)
a = torch.rand(1,100)
b = netG(a)

In [ ]:
# # test netD
# ngpu = 1
# netD = Discriminator(ngpu,verbose=True)
# a = torch.rand(1,1,128,128)
# b = netD(a)

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

netG = Generator(ngpus)
netD = Discriminator(ngpus)

if cuda:
    netG = torch.nn.DataParallel(netG).cuda()
    netD = torch.nn.DataParallel(netD).cuda()

# "Two time-scale update rule"(TTUR) to update netD 4x faster than netG.
optimizerG = optim.Adam(netG.parameters(), lr=args['learning_rate'], betas=(args['beta1'], args['beta2']))
optimizerD = optim.Adam(netD.parameters(), lr=args['learning_rate'], betas=(args['beta1'], args['beta2']))

In [ ]:
# Sample noise used for generated output.
sample_noise = torch.randn(args['sample_size'], latent_dim)
if cuda:
    sample_noise = sample_noise.cuda()
sample_noise_Var = autograd.Variable(sample_noise, requires_grad=False)

# Save config.
LOGGER.info('Saving configurations...')
config_path = os.path.join(model_dir, 'config.json')
with open(config_path, 'w') as f:
    json.dump(args, f)

In [ ]:
LOGGER.info('Loading audio data...')

# Load noise data #
num_ = len(real_datas)
num_valid = int(np.ceil(num_ * args['valid_ratio']))
num_train = num_ - num_valid
train_size = num_train
if not (num_valid > 0 and num_train > 0):
    LOGGER.error("Please download DATASET !")
train_data = batch_generator( real_datas[:num_train],batch_size )
valid_data = batch_generator( real_datas[num_train:],batch_size )

print(f"train:size={train_size}")

In [ ]:
TOTAL_TRAIN_SAMPLES = train_size
BATCH_NUM = TOTAL_TRAIN_SAMPLES // batch_size
train_iter = iter(train_data)
valid_iter = iter(valid_data)
#test_iter = iter(test_data)

In [ ]:
print(next(train_iter)['X'].shape)
print(type(next(train_iter)['X']))

In [ ]:
D_loss = []
G_loss = []



In [ ]:
start = time.time()
LOGGER.info('Starting training...EPOCHS={}, BATCH_SIZE={}, BATCH_NUM={}'.format(epochs, batch_size, BATCH_NUM))
for epoch in range(1, epochs+1):
    LOGGER.info("{} Epoch: {}/{}".format(time_since(start), epoch, epochs))
    
    D_loss_epoch = []
    G_loss_epoch = []
    
    for i in range(1, BATCH_NUM+1):
        
#         # (1) Update D network
#         for p in netD.parameters():
#             p.requires_grad = True
            
        netD.zero_grad()
        
        # latent vector
        noise = torch.Tensor(batch_size, latent_dim).uniform_(-1, 1).view(-1,nz)
        if cuda:
            noise = noise.cuda()
        noise_Var = Variable(noise, requires_grad=False)
        
        
        real_data_Var = numpy_to_var(next(train_iter)['X'], cuda) 
        b_size = real_data_Var.size(0)

        # a) compute loss contribution from real training data
        label = torch.full((b_size,), real_label, dtype=torch.float)
        if cuda:
            label = label.cuda()            

        D_real = netD(real_data_Var).view(-1)

        # Calculate loss on all-real batch
        errD_real = criterion(D_real, label)   

        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = D_real.mean().item()

        # b) compute loss contribution from generated data, then backprop.
        fake = autograd.Variable(netG(noise_Var).data)
        label.fill_(fake_label)
        #print(np.shape(fake))
        D_fake = netD(fake).view(-1)

        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(D_fake, label)

        # Calculate the gradients for this batch
        errD_fake.backward() 

        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake

        # Update D
        optimizerD.step()
        
        if cuda:
            errD = errD.cpu()
            
        # Record loss 
        D_loss_epoch.append(errD.detach().numpy())
        
            
            
        #######################################################
        # (3) Train Generator
        #######################################################
        # Prevent discriminator update.
#         for p in netD.parameters():
#             p.requires_grad = False

        # Reset generator gradients
        netG.zero_grad()
        
        label.fill_(real_label)  # fake labels are real for generator cost
        
        # Noise
        noise = torch.Tensor(batch_size, latent_dim).uniform_(-1, 1).view(-1,nz)
        if cuda:
            noise = noise.cuda()
        noise_Var = Variable(noise, requires_grad=False) 
        
        fake = netD(netG(noise_Var)).view(-1)
        
        # Calculate G's loss based on this output
        errG = criterion(fake, label)
        # Calculate gradients for G
        errG.backward()
        # Update G
        optimizerG.step()
        
        if cuda:
            errG = errG.cpu()
            
        # Record loss 
        G_loss_epoch.append(errG.detach().numpy())
        
        LOGGER.info("{} Epoch={} Batch: {}/{} D_loss{:.4f} | G_loss:{:.4f}".format(time_since(start), epoch,
                                                                                             i, BATCH_NUM,
                                                                                             errD,
                                                                                             errG,
                                                                                             ))    
    
    D_loss_epoch_avg = sum(D_loss_epoch) / float(len(D_loss_epoch))
    G_loss_epoch_avg = sum(G_loss_epoch) / float(len(G_loss_epoch))
    
    D_loss.append(D_loss_epoch_avg)
    G_loss.append(G_loss_epoch_avg)
        
    LOGGER.info("{} D_loss_epoch:{:.4f} |  G_loss_epoch:{:.4f}".format(time_since(start),
                                       D_loss_epoch_avg,
                                       G_loss_epoch_avg,))
    

    if epoch % epochs_per_sample == 0:
        LOGGER.info("Generating samples...")
        sample_out = netG(sample_noise_Var.view(-1,100))  
        if cuda:
            sample_out = sample_out.cpu()
        sample_out = sample_out.data.numpy() 
        sample_out = np.squeeze(sample_out) 
        # save spectrogram and timeseries and statistic
        sample_dir = save_and_plot(sample_out, epoch, output_dir )
        
    if epoch % 20 ==0:
        # Save model
        LOGGER.info("Saving models...")
        netD_path = os.path.join(output_dir, str(epoch) + "discriminator.pkl")
        netG_path = os.path.join(output_dir, str(epoch) + "generator.pkl")
        torch.save(netD.state_dict(), netD_path, pickle_protocol=pickle.HIGHEST_PROTOCOL)
        torch.save(netG.state_dict(), netG_path, pickle_protocol=pickle.HIGHEST_PROTOCOL)
    

    
    

LOGGER.info('>>>>>>>Training finished !<<<<<<<')

# Save model
LOGGER.info("Saving models...")
netD_path = os.path.join(output_dir, "discriminator.pkl")
netG_path = os.path.join(output_dir, "generator.pkl")
torch.save(netD.state_dict(), netD_path, pickle_protocol=pickle.HIGHEST_PROTOCOL)
torch.save(netG.state_dict(), netG_path, pickle_protocol=pickle.HIGHEST_PROTOCOL)


# Plot loss curve.
LOGGER.info("Saving loss curve...")
plot_loss(D_loss, G_loss, output_dir)      



In [ ]:
def plot_loss(D_loss, G_loss, save_path):

    save_path = os.path.join(save_path, "loss_curve.png")

    x = range(len(D_loss))

    y1 = D_loss
    y2 = G_loss

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    plt.savefig(save_path)

In [ ]:
plot_loss(D_loss, G_loss, output_dir)   

In [ ]:
#生成雜訊
LOGGER.info("Generating samples...")
fakee = []
for i in range(470):
    noise = torch.Tensor(batch_size, latent_dim).uniform_(-1, 1).view(-1,100)
    if cuda:
        noise = noise.cuda()
    noise_Var = Variable(noise, requires_grad=False)
    sample_out = netG(noise_Var)  
    if cuda:
        sample_out = sample_out.cpu()
    sample_out = sample_out.data.numpy() 
    sample_out = np.squeeze(sample_out)
    fakee.append(sample_out)
fakee = np.reshape( np.array(fakee),(-1,128,128) )

In [ ]:
print(fakee.shape)

In [ ]:
# a = 'cyclo11024_fake250.csv'
# #fakees = []
# with open(a,'w',newline='') as csvFile:
#     csvWriter = csv.writer(csvFile)
#     for i in range(len(fakee)):
#         if(i%50==0):
#             print(i)
#         csvWriter.writerow( spectrogram_to_time(fakee[i])[1] )   #回傳[1]才是訊號 
#         #fakees.append( spectrogram_to_time(fakee[i])[1] )

In [ ]:
fake_datas = []
for i in range(len(fakee)):
    print(i)
    fake_datas.append( spectrogram_to_time(fakee[i])[1] )
fake_datas = np.array(fake_datas)
print(fake_datas.shape)

In [ ]:
fake_datas = np.float16(fake_datas[:15000])
real_datas = real_datas[:15000]
print(fake_datas.shape)
print(real_datas.shape)

In [ ]:
save = 'metrice'
if not os.path.isdir(save):
    os.makedirs(save)

In [ ]:
a = save + "/" + "dcgan_ep130" + "(cyclo2_data)" + ".csv"
with open(a,'w',newline='') as csvFile:
    csvWriter = csv.writer(csvFile)
    for i in range(len(fake_datas)):
        print(i)
        csvWriter.writerow( fake_datas[i] ) 
        

        
# b = save + "/" + "real" + "(cyclo4_data)" + ".csv"
# with open(b,'w',newline='') as csvFile:
#     csvWriter = csv.writer(csvFile)
#     for i in x:
#         csvWriter.writerow( real_datas[i] )

In [ ]:
# paremeter
fs = 625000 # 400k
data_length = len(real_datas[0])

In [ ]:
# draw time series

import matplotlib.pyplot as plt

t = data_length / fs # sampling freq * sampling time = sample num

plt.figure(figsize=(16,8))
plt.subplot(2,1,1)
plt.plot( np.arange(0,t,1/fs), real_datas[500], label = 'Real',color = (1,0,0))
plt.legend(loc = 'upper right',fontsize=16)
plt.xlabel("Time(s)",fontsize=16)
plt.ylabel("Noise amplitude(v)",fontsize=16)

plt.subplot(2,1,2)
plt.plot( np.arange(0,t,1/fs), fake_datas[2], label = 'Generate')
plt.legend(loc = 'upper right',fontsize=16)
plt.xlabel("Time(s)",fontsize=16)
plt.ylabel("Noise amplitude(v)",fontsize=16)


plt.savefig(save + '/' + "time_series.png")

In [ ]:
x = np.random.randint(1,15000, size=(10))
tot = 1
# draw random 10 time series

import matplotlib.pyplot as plt
#fs = 360000 # 360k
t = data_length / fs # sampling freq * sampling time = sample num

plt.figure(figsize=(15,50))
for i in x:
    plt.subplot(20,1,tot)
    plt.plot( np.arange(0,t,1/fs), real_datas[i], label = 'Real',color = (1,0,0))
    plt.legend(loc = 'upper right',fontsize=16)
    plt.xlabel("Time(s)",fontsize=16)
    plt.ylabel("Noise amplitude(v)",fontsize=10)
    
    tot += 1
    
    plt.subplot(20,1,tot)
    plt.plot( np.arange(0,t,1/fs), fake_datas[i], label = 'Generate')
    plt.legend(loc = 'upper right',fontsize=16)
    plt.xlabel("Time(s)",fontsize=16)
    plt.ylabel("Noise amplitude(v)",fontsize=10)
    
    tot += 1
    
plt.savefig(save + '/' + "time_series(10random).png")

In [ ]:
# draw stft
from scipy import signal
# fs = 360000
f1, t1, Zxx1 = signal.stft(real_datas[0], fs=fs, window='blackmanharris', nperseg=256, noverlap=128)
f2, t2, Zxx2 = signal.stft(fake_datas[0], fs=fs, window='blackmanharris', nperseg=256, noverlap=128)

import matplotlib.pyplot as plt
cm=plt.cm.get_cmap('plasma')
plt.figure(figsize=(20,10))
#plt.title(" X:Time[s], Y:Frequency[KHz]")
plt.subplot(2,1,1)
plt.pcolormesh(t1, f1/1000, 20*np.log10(abs(Zxx1)),cmap=cm)
plt.colorbar()
#plt.legend(loc = 'best',fontsize=16)
plt.xlabel("Time(s)",fontsize=16)
plt.ylabel("Frequency(kHz)",fontsize=16)

plt.subplot(2,1,2)
plt.pcolormesh(t2, f2/1000, 20*np.log10(abs(Zxx2)),cmap=cm)
plt.colorbar()
#plt.legend(loc = 'best',fontsize=16)
plt.xlabel("Time(s)",fontsize=16)
plt.ylabel("Frequency(kHz)",fontsize=16)
plt.savefig(save + '/' + "stft.png")

In [ ]:
from scipy.fftpack import fft,ifft
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
le = len(real_datas[2])
xf = np.linspace(0,fs,data_length) # 頻率
xf2 = xf[:(le//2)]
r_yf2 = abs(fft(fake_datas[100]))[:(le//2)]
plt.plot(xf2,r_yf2)
print(r_yf2.shape)
print(xf2)

In [ ]:
# Average FFT

from scipy.fftpack import fft,ifft
r_average_fft = np.zeros(len(real_datas[0]))
f_average_fft = np.zeros(len(fake_datas[0]))
le = len(real_datas)
for i in range(len(real_datas)):
    r_average_fft += 20*np.log10(abs( fft(real_datas[i])/16384 )) / le
    f_average_fft += 20*np.log10(abs( fft(fake_datas[i])/16384 )) / le
    
ln = len(real_datas[0])
import matplotlib.pyplot as plt
r_yf2 = r_average_fft[ : (ln//2) ]
f_yf2 = f_average_fft[ : (ln//2) ]
xf = np.linspace(0,fs,data_length) # 頻率
xf2 = xf[:(ln//2)] / 1000

#print( r_average_fft.shape)

plt.figure(figsize=(16,8))
plt.plot(xf2,r_yf2,label = 'Real',color=(1,0,0))
plt.plot(xf2,f_yf2,label = 'Generate',alpha=0.5)
plt.legend(loc='best',fontsize=16)
plt.title('FFT of Mixed wave',fontsize=20,color='#F08080')
plt.xlabel("Frequency(kHz)",fontsize=16)
plt.ylabel("Average FFT(db)",fontsize=16)

plt.savefig(save + '/' + "fft.png")

In [ ]:
# statisitc

from scipy import stats
from scipy.signal import find_peaks

def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size//2:]

def feature_change(chandata):
    total_data = []
    print(len(chandata))
    for i in range(len(chandata)):
        print(i)
        # statistic data
        nobs,(damin,damax),damean,davar,daskew,dakurto = stats.describe( chandata[i] )
        # energy
        E_fft = np.sum(chandata[i] ** 2) / len(chandata[i])
        # auto statistic data
        nobs,(aumin,aumax),aumean,auvar,auskew,aukurto = stats.describe( autocorr(chandata[i]) ) 
        # peak > 0.05
        peaks, _ = find_peaks(chandata[i], height=0.05)
        
        total_data.append( [damax, damean*1000, E_fft, davar**(1/2), daskew, dakurto, int(len(peaks)), auskew, aukurto] )
    return np.array( total_data )

rdata_statis = feature_change(real_datas)
fdata_statis = feature_change(fake_datas)

print(rdata_statis.shape)
print(fdata_statis.shape)

In [ ]:
# store random 100 datas to generate CPS and coherence

a = save + "/" + "dcgan_ep130_statistic" + "(cyclo4_data)" + ".csv"
with open(a,'w',newline='') as csvFile:
    csvWriter = csv.writer(csvFile)
    for i in range(len(fdata_statis)):
        csvWriter.writerow( fdata_statis[i] ) 
        

        
b = save + "/" + "real_statistic" + "(cyclo4_data)" + ".csv"
with open(b,'w',newline='') as csvFile:
    csvWriter = csv.writer(csvFile)
    for i in range(len(rdata_statis)):
        csvWriter.writerow( rdata_statis[i] )

In [ ]:
# real datas

total_mean = np.mean(rdata_statis, axis = 0)
total_std = np.std(rdata_statis, axis = 0)
total_median = np.median(rdata_statis, axis = 0)

print("max[v], mean[mv], energy[J], std[v], skewness, kurtosis, peak>0.05, skewauto, kuroauto")
print(total_mean)
print(total_std)
print(total_median)

with open(save + '/' + 'real_statistic.csv','w',newline="") as csvfile:
    outwriter = csv.writer(csvfile)
    outwriter.writerow(['feature']+['max[v]', 'mean[mv]', 'energy[J]', 'std[v]', 'skewness', 'kurtosis', 'peak>0.05', 'skewauto', 'kuroauto'])
    outwriter.writerow(['mean']+list(total_mean))
    outwriter.writerow(['std']+list(total_std))
    outwriter.writerow(['median']+list(total_median))
    csvfile.close()
    

#fake datas

total_mean = np.mean(fdata_statis, axis = 0)
total_std = np.std(fdata_statis, axis = 0)
total_median = np.median(fdata_statis, axis = 0)

print("max[v], mean[mv], energy[J], std[v], skewness, kurtosis, peak>0.05, skewauto, kuroauto")
print(total_mean)
print(total_std)
print(total_median)
    
with open(save + '/' + 'fake_statistic.csv','w',newline="") as csvfile:
    outwriter = csv.writer(csvfile)
    outwriter.writerow(['feature']+['max[v]', 'mean[mv]', 'energy[J]', 'std[v]', 'skewness', 'kurtosis', 'peak>0.05', 'skewauto', 'kuroauto'])
    outwriter.writerow(['mean']+list(total_mean))
    outwriter.writerow(['std']+list(total_std))
    outwriter.writerow(['median']+list(total_median))
    csvfile.close()

In [ ]:
# normalize data

#r_norm = np.linalg.norm(rdata_statis,axis=0)
r_max = np.max(rdata_statis,axis=0)
r_min = np.min(rdata_statis, axis = 0)
r_mean = np.mean(rdata_statis, axis = 0)
rdata_statis_norm = np.zeros(np.shape(rdata_statis))
for i in range(rdata_statis_norm.shape[1]):
    rdata_statis_norm[:,i] = ( (rdata_statis[:,i]-r_mean[i]) )/(r_max[i]-r_min[i])


#f_norm = np.linalg.norm(fdata_statis,axis=0)
f_max = np.max(fdata_statis,axis=0)
f_min = np.min(fdata_statis, axis = 0)
f_mean = np.mean(fdata_statis, axis = 0)
fdata_statis_norm = np.zeros(np.shape(fdata_statis))
for i in range(fdata_statis_norm.shape[1]):
    fdata_statis_norm[:,i] = ( (fdata_statis[:,i]-f_mean[i]) )/(f_max[i]-f_min[i])

print(rdata_statis_norm.shape)
print(fdata_statis_norm.shape)


# mix

dataplot = []
for i in range(9):
    dataplot.append(rdata_statis_norm[:,i])
    dataplot.append(fdata_statis_norm[:,i])
dataplot = np.transpose(np.array(dataplot))
print(dataplot.shape)

labels=[]
for i in range(1,10):
    labels.append(str(i)+"")
    labels.append('*')

fig,axes = plt.subplots()
axes.boxplot(x=dataplot,sym='rd',patch_artist=True,labels=labels)
axes.set_xlabel('Feature')
plt.text(14,0.90,'Num : Real')
plt.text(15,0.80,'* : Generate')
plt.savefig(save + '/' + "boxplot.png")

In [ ]:
# normalize data

#r_norm = np.linalg.norm(rdata_statis,axis=0)
r_max = np.max(rdata_statis,axis=0)
r_min = np.min(rdata_statis, axis = 0)
r_mean = np.mean(rdata_statis, axis = 0)
rdata_statis_norm = np.zeros(np.shape(rdata_statis))
for i in range(rdata_statis_norm.shape[1]):
    maxx = abs(r_max[i]) if abs(r_max[i])>abs(r_min[i]) else abs(r_min[i])
    rdata_statis_norm[:,i] =  rdata_statis[:,i] /maxx


#f_norm = np.linalg.norm(fdata_statis,axis=0)
f_max = np.max(fdata_statis,axis=0)
f_min = np.min(fdata_statis, axis = 0)
f_mean = np.mean(fdata_statis, axis = 0)
fdata_statis_norm = np.zeros(np.shape(fdata_statis))
for i in range(fdata_statis_norm.shape[1]):
    maxx = abs(f_max[i]) if abs(f_max[i])>abs(f_min[i]) else abs(f_min[i])
    fdata_statis_norm[:,i] =  fdata_statis[:,i] /maxx
print(rdata_statis_norm.shape)
print(fdata_statis_norm.shape)


# mix

dataplot = []
for i in range(9):
    dataplot.append(rdata_statis_norm[:,i])
    dataplot.append(fdata_statis_norm[:,i])
dataplot = np.transpose(np.array(dataplot))
print(dataplot.shape)

labels=[]
for i in range(1,10):
    labels.append(str(i)+"")
    labels.append('*')

fig,axes = plt.subplots()
axes.boxplot(x=dataplot,sym='rd',patch_artist=True,labels=labels)
axes.set_xlabel('Feature')
plt.text(14,-0.8,'Num : Real')
plt.text(15,-0.9,'* : Generate')
plt.savefig(save + '/' + "boxplot_1.png")

In [ ]:
print(r_min)